<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BC%D0%BE%D0%BB%D0%B5%D0%BA%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%83%D1%8E_%D0%B1%D0%B8%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%8E_%D0%B8_%D0%B3%D0%B5%D0%BD%D0%B5%D1%82%D0%B8%D0%BA%D1%83_%D0%BF%D1%80%D0%BE%D0%BC%D0%B5%D0%B6%D1%83%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B0%D1%82%D1%82%D0%B5%D1%81%D1%82%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
import xml.etree.ElementTree as ET
from pprint import pprint

import pandas as pd
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.6 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/GB/19_6_files'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/GB/19_6_files'
/content


Получаем список генов по наименованию болезни из базы NCBI

In [5]:
def get_genes_by_disease(disease, retmax):
  disease_name = disease #Вносим название болезни во внутреннюю переменную
  handle = Entrez.esearch(db="gene", term=disease_name, retmax = retmax) #'retmax' Ограничиваем количество генов для вывода
  record = Entrez.read(handle)
  gene_ids = record["IdList"]
  return gene_ids

In [6]:
#Запрашиваем ID хромосомы из NCBI через параметр 'gi_id'.
#Эта процедура применяется в общей процедуре сбора информации по генам(см. ячейку ниже)
def search_chromosome(gi_id):
    handle = Entrez.esummary(db="nucleotide", id=gi_id)
    record = Entrez.read(handle)
    handle.close()
    return record[0]['AccessionVersion']

In [7]:
# Поиск в базе по имени гена
def search_gene_name(gene_id):
  handle = Entrez.esummary(db="gene", id=gene_id)
  record = Entrez.read(handle)
  handle.close()
  return record['DocumentSummarySet']['DocumentSummary'][0]['Name']

In [8]:
#Получаем из NCBI информацию о генах из списка 'gene_id_list', в том числе ID хромосомы.
def get_gene_info(gene_id_list):
  gene_info = []
  for gene_id in gene_id_list:
    handle = Entrez.efetch(db="gene", id=gene_id, rettype = 'xml')
    gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
    region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
    start = int(region.find("Seq-interval_from").text) + 1
    end = int(region.find("Seq-interval_to").text) + 1
    gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
    strand = region.find("Seq-interval_strand/Na-strand").get("value")
    # print(gene_id, start, end, gi_id, strand)
    gene_info.append({'gene_name':search_gene_name(gene_id), 'gene_id':gene_id, 'interval_from':start, 'interval_to':end, 'gi_id':gi_id, 'nucleotide':search_chromosome(gi_id), 'strand':strand})
  return gene_info

In [9]:
%%time
gene_id_list = get_genes_by_disease('Apert syndrome', 30)
gene_list = get_gene_info(gene_id_list)

IncompleteRead: IncompleteRead(38611 bytes read)

In [10]:
#Предоставляем список генов в виде таблицы
df = pd.DataFrame(gene_list)
df['accession'] = df['nucleotide'].apply(lambda x: x.split('.')[0])
df['version'] = df['nucleotide'].apply(lambda x: x.split('.')[1])

NameError: name 'gene_list' is not defined

In [11]:
df.T

NameError: name 'df' is not defined

In [ ]:
#Получаем нуклеотидную последовательность гена. Параметры берем из 'gene_list'
def get_dna_sequence(gene_seq_keys_list):
  gene_seq_list = []
  for nucleotid in gene_seq_keys_list:
    nucleotid_id = nucleotid['nucleotide']
    start = nucleotid['interval_from']
    end = nucleotid['interval_to']
    handle = Entrez.efetch(db="nucleotide",
                        id=nucleotid_id,
                        rettype = 'fasta',
                        retmode = 'text')
    #загружаем нуклеотидную последовательность хромосомы
    chromosome = SeqIO.read(handle, 'fasta')
    gene_seq = chromosome.seq[start:end]
    gene_seq_list.append({'Gene ID': nucleotid["gene_id"], 'gen_seq': gene_seq})
  return gene_seq_list

In [ ]:
%%time
gene_seq_list = get_dna_sequence(gene_list)
pprint(gene_seq_list)

In [ ]:
# Получаем информацию из базы данных SNP для гена FGFR2
gene_name = "FGFR2"
org = "homo sapiens"
# search ClinVar for variants in the gene
handle = Entrez.esearch(db="snp", term=f"{gene_name}[Gene Name]  AND {org}[Organism] AND (pathogenic[Clinical_Significance] OR likely pathogenic[Clinical_Significance] OR pathogenic likely pathogenic[Clinical_Significance])", retmax=100)
# handle = Entrez.esearch(db="snp", term=f"{gene_name}[Gene Name]  AND {org}[Organism]", retmax=60000)
record = Entrez.read(handle)
handle.close()
rs_list = record['IdList']
len(rs_list)

In [ ]:
# Получаем информацию по каждой мутации из списка
def get_snp_rs_info(rs_list):
  snp_rs_info = []
  for k, i in enumerate(rs_list):
    print(f'\r{k} ' , end='')
    handle = Entrez.esearch(db="snp", term="rs" + i, format='gb')
    #handle = Entrez.efetch(db="snp", id="rs" + i)
    record = Entrez.read(handle)
    handle.close()
    idlist = record["IdList"]

    if len(idlist) > 0:
        handle = Entrez.esummary(db="snp", id=",".join(idlist))
        result = Entrez.read(handle)
        handle.close()
        snp_info = result['DocumentSummarySet']['DocumentSummary'][0]
        snp_rs_info.append({key: snp_info[key] for key in snp_info.keys()})
        # snp_rs_info.append({'SNP_ID':f"rs{snp_info['SNP_ID']}", 'GENES':snp_info['GENES'], 'ACC':snp_info['ACC'], 'SPDI':snp_info['SPDI'], 'ALLELE':" > ".join(snp_info['SPDI'].split(':')[2:]), 'CLINICAL_SIGNIFICANCE':snp_info['CLINICAL_SIGNIFICANCE'], 'CLINICAL_SORT':snp_info['CLINICAL_SORT']})
  return snp_rs_info

In [ ]:
# search for the SNP record in NCBI dbSNP database
handle = Entrez.esearch(db="snp", term="rs77543610")
record = Entrez.read(handle)
print(record)
snp_rs_info=[]
# retrieve the relevant data for the SNP
handle = Entrez.esummary(db="snp", id=record["IdList"][0], rettype="asn.1")
record = Entrez.read(handle)
#snp_info = record['DocumentSummarySet']['DocumentSummary'][0]
#snp_rs_info.append({'SPDI':snp_info['SPDI']})
#snp_rs_info.append({'ALLELE':" > ".join(snp_info['SPDI'].split(':')[2:])})
pprint(record)

1. Оценить частоту мутаций в популяциях по выбранному гену

In [ ]:
df = pd.read_parquet('M4team_dataset.parquet.gzip')

In [ ]:
df_new = df.copy()

In [ ]:
df_new.head(2)

In [ ]:
df_new.info()

In [ ]:
sign_list = ['pathogenic',
       'pathogenic,likely-pathogenic,pathogenic-likely-pathogenic',
       'pathogenic,likely-pathogenic,uncertain-significance,benign-likely-benign',
       'likely-pathogenic,uncertain-significance,pathogenic',
       'likely-pathogenic',
       'likely-pathogenic,pathogenic',
       'pathogenic,uncertain-significance']

In [ ]:
df_new.columns

In [ ]:
df_new.drop(columns=['SNP_ID',  'GLOBAL_SAMPLESIZE', 'CHR'], inplace=True)

In [ ]:
df_new.CLINICAL_SIGNIFICANCE.value_counts()

In [ ]:
# rs_pat_list = df_fgfr2_pat.SNP_ID_RS.values.tolist()

In [ ]:
df_new.Population.value_counts()

In [ ]:
# Унификация названий популяции
d = {'America' : 'American', 'Europe' : 'European', 'Africa' : 'African',
     'AfricanAmerican': 'African American', 'Est_Asia': 'East Asian',
     'SouthAsian': 'South Asian'}
df_new['Population'] = df_new['Population'].apply(lambda x: d[x] if x in d.keys() else x)

# Удаление некоторых категорий
df_new = df_new.loc[~df_new['Population'].isin(['PARENT AND CHILD COHORT', 'TWIN COHORT', 'ACPOP', 'Other', 'Total', 'Global', 'CRM'])]

In [ ]:
# df_new = df_new[df['Ref Allele'] !='-\nNo frequency provided']
df_new['Alt_Allele'] = df_new['Alt Allele'].apply(lambda x: x.split('=')[0])
df_new['Alt_Allele_frequency'] = df_new['Alt Allele'].apply(lambda x: x.split('=')[1].split(',')[0]).astype(float)

In [ ]:
# Удаление строк, где Samplesize = 0
df_new = df_new[df_new['Samplesize']!= 0]

In [ ]:
df_new.loc[df_new['Samplesize']== 0]

In [ ]:
# Отфильтруем данные по CLINICAL_SIGNIFICANCE = pathogenic
df1_new = df_new.loc[df_new['CLINICAL_SIGNIFICANCE'].isin(['pathogenic'])]

In [ ]:
df1_new.groupby(['Population', 'Alt_Allele']).agg({'Alt_Allele_frequency': ['max', 'mean', 'sum'], 'Samplesize': ['max', 'sum']}).reset_index()

In [ ]:
df1_new.Population.value_counts()

In [ ]:
fig, ax = plt.subplots()
for title, group in df1_new.groupby('Population'):
    ax.scatter(group['Alt_Allele'], group['Alt_Allele_frequency'], label=title)

plt.xlabel('Alt_Allele')
plt.ylabel('Alt_Allele_frequency')
plt.title('pop and Alt_Allele_frequency')

plt.legend()
plt.show()

In [ ]:
# Посмотрим частоту альтернативных аллелей в пределах одной популяции
df_american = df1_new.loc[df1_new['Population'].isin(['American']) & df1_new['Alt_Allele'].isin(['A', 'T', 'G', 'C'])]

In [ ]:
df_american.groupby(['Population', 'Alt_Allele']).agg({'Alt_Allele_frequency': ['max', 'mean', 'sum'], 'Samplesize': ['max', 'sum']}).reset_index()

In [ ]:
fig, ax = plt.subplots()

for title, group in df_american.groupby('Population'):
    ax.scatter(group['Alt_Allele'], group['Alt_Allele_frequency'], label=title)

plt.xlabel('Alt_Allele')
plt.ylabel('Alt_Allele_frequency')
plt.title('American and Alt_Allele_frequency')

plt.legend()
plt.show()

In [ ]:
df_european = df1_new.loc[df1_new['Population'].isin(['European']) & df1_new['Alt_Allele'].isin(['A', 'T', 'G', 'C'])]

In [ ]:
df_european.groupby(['Population', 'Alt_Allele']).agg({'Alt_Allele_frequency': ['max', 'mean', 'sum'], 'Samplesize': ['max', 'sum']}).reset_index()

In [ ]:
fig, ax = plt.subplots()

for title, group in df_european.groupby('Population'):
    ax.scatter(group['Alt_Allele'], group['Alt_Allele_frequency'], label=title)

plt.xlabel('Alt_Allele')
plt.ylabel('Alt_Allele_frequency')
plt.title('European and Alt_Allele_frequency')

plt.legend()
plt.show()

2. Оценить частоту мутаций в разрезе связанных заболеваний.

In [ ]:
df2 = pd.read_parquet('FGFR2_disease_dataset.parquet.gzip')

In [ ]:
df_dis = df2.copy()
df_dis.head(2)

In [ ]:
df_dis.info()

In [ ]:
df_dis.drop(columns=['ALLELE_ORIGIN', 'GLOBAL_SAMPLESIZE', 'GLOBAL_POPULATION', 'CHR', 'SUSPECTED', 'TEXT', 'CITED_SORT' ], inplace=True)

In [ ]:
df_dis.columns

In [ ]:
df_dis.Disease_Names.value_counts()

In [ ]:
# Унификация названий болезней
d = {'Craniosynostosis, nonspecific' : 'FGFR2-related syndromic and non-syndromic craniosynostoses',
     'Craniosynostosis syndrome' : 'FGFR2-related syndromic and non-syndromic craniosynostoses',
     'Craniosynostosis, nonsyndromic unicoronal' : 'FGFR2-related syndromic and non-syndromic craniosynostoses',
     'Craniosynostosis, nonclassifiable autosomal dominant': 'FGFR2-related syndromic and non-syndromic craniosynostoses',
     'FGFR2 related craniosynostosis': 'FGFR2-related syndromic and non-syndromic craniosynostoses',
     'Pfeiffer syndrome variant': 'Pfeiffer syndrome',
     'Pfeiffer syndrome type 3': 'Pfeiffer syndrome',
     'Squamous cell lung carcinoma': 'Lung adenocarcinoma',
     'Abnormal pinna morphology,Abnormal posterior cranial fossa morphology,Abnormal zygomatic bone morphology,Brachyturricephaly,Choanal stenosis,Cranial asymmetry,Deviated nasal septum,Dilation of lateral ventricles,Downslanted palpebral fissures,Facial asymmetry,Flat occiput,High forehead,High palate,Hydrocephalus,Hypertelorism,Hypointensity of cerebral white matter on MRI,Low-set ears,Mild fetal ventriculomegaly,Narrow forehead,Shallow orbits,Short neck,Wide anterior fontanel':
     'Craniofacial-skeletal-dermatologic dysplasia',
     'Acrocephalosyndactyly type I,Antley-Bixler syndrome without genital anomalies or disordered steroidogenesis,Beare-Stevenson cutis gyrata syndrome,Crouzon syndrome,FGFR2-related bent bone dysplasia,Familial scaphocephaly syndrome, McGillivray type,Jackson-Weiss syndrome,Levy-Hollister syndrome,Neoplasm of stomach,Pfeiffer syndrome,Saethre-Chotzen syndrome':
     'Undefined FGFR2-related syndrome',
     'Acrocephalosyndactyly type I': 'Apert syndrome',
     'not provided': '[Not provided]'}
df_dis['Disease_Names'] = df_dis['Disease_Names'].apply(lambda x: d[x] if x in d.keys() else x)

# Удаление некоторых категорий
# df_dis = df_dis.loc[~df_dis['Disease_Names'].isin(['not provided'])]

In [ ]:
df_dis.Disease_Names.value_counts()

In [ ]:
df_dis.SNP_CLASS.value_counts()

In [ ]:
fig, ax = plt.subplots()

for title, group in df_dis.groupby('Disease_Names'):
    ax.scatter(group['SNP_CLASS'], group['Disease_Names'])

fig.set_figheight(7)
fig.set_figwidth(4)

# ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

plt.xlabel('SNP class')
plt.ylabel('Disease Names')
plt.title('SNP class distribution of FGFR2-related diseases')

# plt.legend()
plt.show()

In [ ]:
df_dis['Mutation_type'].value_counts()

In [ ]:
# Унификация названий типов мутаций
d = {'Missense Variant' : 'Missense',
     'Stop Gained' : 'Nonsense (Stop Gained)',
     'Synonymous Variant' : 'Synonymous',
     'Splice Acceptor Variant' : 'Splice Acceptor',
     'Splice Donor Variant' : 'Splice Donor',
     'Intron Variant' : 'Intron'}
df_dis['Mutation_type'] = df_dis['Mutation_type'].apply(lambda x: d[x] if x in d.keys() else x)

In [ ]:
df_dis['Mutation_type'].value_counts()

In [ ]:
fig, ax = plt.subplots()

for title, group in df_dis.groupby('Disease_Names'):
    ax.scatter(group['Mutation_type'], group['Disease_Names'])

fig.set_figheight(7)
fig.set_figwidth(4)

ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha='right')

plt.xlabel('Mutation type')
plt.ylabel('Disease Names')
plt.title('Mutation types of FGFR2-related diseases')

# plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

for title, group in df_dis.groupby('Mutation_type'):
    ax.scatter(group['SNP_CLASS'], group['Mutation_type'])

fig.set_figheight(4)
fig.set_figwidth(4)

# ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha='right')

plt.xlabel('SNP_CLASS')
plt.ylabel('Mutation type')
plt.title('Mutation types and SNP class relation of FGFR2-related diseases')

# plt.legend()
plt.show()